In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
def team_name(filename):
    df = pd.read_csv(filename)
    df["team"] = filename.split("/")[1].split("-schedule")[0]

def extract_opponent(game_info):
    if '@' in game_info:
        if "" in (game_info.split("@")):
            return game_info.split("@")[1].strip()
        else:
            return game_info.split("@")[0].strip()
    else:
        return game_info

def game_site(game_info):
    if '@' in game_info:
        if "" in (game_info.split("@")):
            return "away"
        else:
            return "neutral"
    else:
        return "home"
def date_conversion(date_string):
    return datetime.strptime(date_string, "%m/%d/%Y").date()


In [3]:
df_list = []
for directory in os.listdir("all_schedules"):
    for file in os.listdir("all_schedules/"+directory):
        filename = os.path.join("all_schedules", directory, file)
        df = pd.read_csv(filename)
        df["team"] = file.split("-schedule")[0]
        df["opponent"] = df["opponent/venue"].apply(extract_opponent)
        df["game_site"] = df["opponent/venue"].apply(game_site)
        df["date"] = df["date"].apply(date_conversion)
        df_list.append(df)

result_df = pd.concat(df_list, ignore_index=True)

In [4]:
len(result_df)

137882

In [5]:
#TODO: drop mp, attend, retatt, ms, tb
result_df.head()

,date,opponent/venue,result,sets,mp,kills,errors,total_attacks,hit_pct,assists,...,b_error,pts,bhe,team,opponent,game_site,retatt,attend,ms,tb
0,2016-08-26,"Florida @ Eugene, Ore.",3-1,4.0,0.0,67.0,12.0,143.0,0.385,64.0,...,3.0,81.5,0.0,Arizona,Florida,neutral,NaN,NaN,NaN,NaN
1,2016-08-27,"Texas @ Eugene, Ore.",3-0,3.0,0.0,48.0,14.0,112.0,0.304,42.0,...,1.0,62.0,1.0,Arizona,Texas,neutral,NaN,NaN,NaN,NaN
2,2016-09-02,Iowa St.,3-0,3.0,0.0,43.0,8.0,90.0,0.389,36.0,...,2.0,60.5,0.0,Arizona,Iowa St.,home,NaN,NaN,NaN,NaN
3,2016-09-03,Oregon St.,3-0,3.0,0.0,41.0,10.0,82.0,0.378,35.0,...,3.0,54.0,0.0,Arizona,Oregon St.,home,NaN,NaN,NaN,NaN
4,2016-09-09,"Arkansas @ Albuquerque, N.M.",3-0,3.0,0.0,40.0,15.0,98.0,0.255,37.0,...,0.0,49.0,0.0,Arizona,Arkansas,neutral,NaN,NaN,NaN,NaN
